In [ ]:
# default_exp ensembl

# ensembl

> Query sequence data in ensembl

In [ ]:
# export
import requests
import json

## Query the ensembl database
Functions to query https://rest.ensembl.org/

### GET

In [ ]:
# export
def ensembl_get(ext, query=None, headers=None, params=None):
    """Generic wrapper for using GET requests to the ensembl rest API

    ext: str, url extension |
    query: str or None, end of url extension specifying species, taxon, esnembl_id etc |
    headers: dict or None,  meta-information for query |
    params: dict or None, parameters for query |

    returns: Response object
    """
    if query is None:
        query = ''
    if params is None:
        params = {}
    if headers is None:
        headers = {}
    r = requests.get("https://rest.ensembl.org"+ext+query, params=params, headers=headers)
    if not r.ok:
        r.raise_for_status()
    else:
        return r

### POST

In [ ]:
# export
def ensembl_post(ext, data, headers=None, params=None):
    """Generic wrapper for using POST requests to the ensembl rest API

    ext: str, url extension |
    data: dict, query data |
    headers: dict or None,  meta-information for query |
    params: dict or None, parameters for query |

    returns: Response object
    """
    if params is None:
        params = {}
    if headers is None:
        headers = {}
    data = json.dumps(data)
    r = requests.post("https://rest.ensembl.org"+ext, headers=headers, data=data, params=params)
    if not r.ok:
        r.raise_for_status()
    else:
        return r

## Get Transcript Meta-information

In [ ]:
#export
def get_ensembl_id_information(ensembl_id, expand=1, utr=1, **kwargs):
    """Get information for an ensembl id (ref. https://rest.ensembl.org/documentation/info/lookup)

    ensembl_id: ENSG or ENST |
    expand: Expands the search to include any connected features.
        e.g. If the object is a gene, its transcripts, translations and exons will be returned as well. |
    utr: Include 5' and 3' UTR features. Only available if the expand option is used. |
    kwargs: other parameters for lookup |

    returns: dict
    """
    headers = {'content-type': 'application/json'}
    params = {'expand': expand, 'utr': utr, **kwargs}
    r = ensembl_get('/lookup/id/', query=ensembl_id,
                    params=params, headers=headers)
    transcript_info = r.json()
    return transcript_info

In [ ]:
assert get_ensembl_id_information('ENSG00000157764')['display_name'] == 'BRAF'

## Get sequence by transcript

Note that `get_transcript_sequence` returns sequences 5' to 3' on same strand as the target

In [ ]:
# export
def get_transcript_sequence(ensembl_id, seq_type='cds',
                            **kwargs):
    """Request multiple types of sequence by stable identifier. Supports feature masking and expand options.
    Uses https://rest.ensembl.org/documentation/info/sequence_id

    seq_type: str, one of [genomic, cds, cdna, protein] |
    kwargs: additional parameter arguments |

    returns: str, sequence 5' to 3' in the same orientation as the input transcript
    """
    headers = {'content-type': 'text/plain'}
    params = {'type': seq_type, **kwargs}
    r = ensembl_get("/sequence/id/", query=ensembl_id, params=params,
                    headers=headers)
    seq = r.text
    return seq

In [ ]:
assert(get_transcript_sequence('ENSE00001154485', seq_type='genomic', mask_feature=0,
                               expand_5prime=10) ==
       'CCCTCCCCGCCCGACAGCGGCCGCTCGGGCCCCGGCTCTCGGTTATAAGATGGCGGCGCTGAGCGGTGGCGGTGGTGGCGGCGCGGAGCCGGGCCAGGCTCTGTTCAACGGGGACATGGAGCCCGAGGCCGGCGCCGGCGCCGGCGCCGCGGCCTCTTCGGCTGCGGACCCTGCCATTCCGGAGGAG')

## Post sequence by transcript

In [ ]:
# export
def post_transcript_sequence(ensembl_ids, seq_type='cdna',
                            **kwargs):
    """Request multiple types of sequence by stable identifier. Supports feature masking and expand options.
    Uses https://rest.ensembl.org/documentation/info/sequence_id

    seq_type: str, one of [genomic, cds, cdna, protein] |
    kwargs: additional parameter arguments |

    returns: str, sequence 5' to 3' in the same orientation as the input transcript
    """
    headers={"content-type" : "application/json", "accept" : "application/json"}
    params = {'type': seq_type, **kwargs}
    data = {'ids': ensembl_ids}
    r = ensembl_post("/sequence/id/", data=data, params=params,
                     headers=headers)
    seq_list = r.json()
    return seq_list

In [ ]:
assert(post_transcript_sequence(["ENSG00000157764", "ENSG00000248378"],
                                seq_type='genomic')[0]['seq'][:4] == 'CTTC')

## Get Sequence by region

Note that `get_region_sequence` returns sequences 5' to 3' on the positive DNA strand

In [ ]:
# export
def create_region_str(start, end, seq_region):
    """Generate string to query a geonomic region of interest

    start: int |
    end: int |
    seq_region: str, chromosome |

    returns: str
    """
    region = seq_region + ':' + str(start) + '..' + str(end)
    return region


def get_region_sequence(start, end, seq_region, **kwargs):
    """Returns the genomic sequence of the specified region of the given species.
    Supports feature masking and expand options.
    Uses https://rest.ensembl.org/documentation/info/sequence_region

    start: int, genomic start position |
    end: int, genomic end position |
    seq_region: str, chromosome |
    kwargs: additional arguments to send to query |

    returns: str, sequence 3' to 5' on the positive DNA strand
    """
    region = create_region_str(start, end, seq_region)
    headers = {'content-type': 'text/plain'}
    params = {**kwargs}
    r = ensembl_get('/sequence/region/human/', query=region, headers=headers,
                    params=params)
    sequence = r.text
    return sequence

In [ ]:
assert (get_region_sequence(1000000, 1000100, 'X') ==
        'CTGTAGAAACATTAGCCTGGCTAACAAGGTGAAACCCCATCTCTACTAACAATACAAAATATTGGTTGGGCGTGGTGGCGGGTGCTTGTAATCCCAGCTAC')

## Post sequence by region

In [ ]:
# export
def post_region_sequences(start, end, seq_region, **kwargs):
    """Request multiple types of sequence by a list of regions.
    Uses https://rest.ensembl.org/documentation/info/sequence_region_post

    start: list of int, start of each query region. Should align with end and seq_region |
    end: list of int |
    seq_region: list of str |
    kwargs: optional params for the query |

    returns: list of dict
    """
    regions = []
    for s, e, r in zip(start, end, seq_region):
        region_str = create_region_str(s, e, r)
        regions.append(region_str)
    ext = "/sequence/region/human"
    headers={"content-type" : "application/json", "accept" : "application/json"}
    params = {**kwargs}
    regions_dict = {'regions': regions}
    r = ensembl_post(ext, data=regions_dict, headers=headers, params=params)
    seq_list = r.json()
    return seq_list

In [ ]:
assert (post_region_sequences([1000000, 1], [1000100, 100],
                              ['X', 'ABBA01004489.1'])[0]['seq'] ==
        'CTGTAGAAACATTAGCCTGGCTAACAAGGTGAAACCCCATCTCTACTAACAATACAAAATATTGGTTGGGCGTGGTGGCGGGTGCTTGTAATCCCAGCTAC')